In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import make_friedman1
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, root_mean_squared_error, r2_score

#### Load data set

In [15]:
df = pd.read_csv("FloridaBikeRentals.csv")
df.head(10)

for col in df.columns:
    data_type = df[col].dtype
    if data_type == object:
        data_values = df[col].unique()
        print(f'Feature Name: {col} Data Type: {data_type} Values: {data_values[:20]}')
    else:
        max_data = df[col].min()
        min_data = df[col].max()
        print(f'Feature Name: {col} Data Type: {data_type} Min: {min_data} Max: {max_data}')

Feature Name: Date Data Type: object Values: ['01/12/2017' '02/12/2017' '03/12/2017' '04/12/2017' '05/12/2017'
 '06/12/2017' '07/12/2017' '08/12/2017' '09/12/2017' '10/12/2017'
 '11/12/2017' '12/12/2017' '13/12/2017' '14/12/2017' '15/12/2017'
 '16/12/2017' '17/12/2017' '18/12/2017' '19/12/2017' '20/12/2017']
Feature Name: Rented Bike Count Data Type: int64 Min: 3556 Max: 0
Feature Name: Hour Data Type: int64 Min: 23 Max: 0
Feature Name: Temperature(C) Data Type: float64 Min: 39.4 Max: -17.8
Feature Name: Humidity(%) Data Type: int64 Min: 98 Max: 0
Feature Name: Wind speed (m/s) Data Type: float64 Min: 7.4 Max: 0.0
Feature Name: Visibility (10m) Data Type: int64 Min: 2000 Max: 27
Feature Name: Dew point temperature(C) Data Type: float64 Min: 27.2 Max: -30.6
Feature Name: Solar Radiation (MJ/m2) Data Type: float64 Min: 3.52 Max: 0.0
Feature Name: Rainfall(mm) Data Type: float64 Min: 35.0 Max: 0.0
Feature Name: Snowfall (cm) Data Type: float64 Min: 8.8 Max: 0.0
Feature Name: Seasons Data 